**Task**

Take in T/R cases and output either T or R

**Aproaches**

1. Used tuned GlossBert and next sentence prediction, and finetune a classifier.
2. 1-NN or Cosine similarity - Sentence level vs Word level

In [ ]:
!pip install transformers
import torch
from transformers import BertTokenizer, BertForNextSentencePrediction

import matplotlib.pyplot as plt
% matplotlib inline

!pip install py_babelnet
from py_babelnet.calls import BabelnetAPI

     |████████████████████████████████| 1.1MB 2.8MB/s 
     |████████████████████████████████| 1.1MB 18.6MB/s 
     |████████████████████████████████| 890kB 16.5MB/s 
     |████████████████████████████████| 3.0MB 27.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=862eabb70ed19b2328c933cb2e323fe6e764ea95738d0543b55f57289035118d
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
api = BabelnetAPI('98da6c4d-1cd8-4367-a2f7-0ef488a8a12a')

In [ ]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

model = BertForNextSentencePrediction.from_pretrained('bert-base-multilingual-cased', return_dict = True)

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForNextSentencePrediction(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [ ]:
# define a function that will return a set of glosses for a given lemma
def babelnet_glosses(lemma_, pos_, api):
  ids = api.get_synset_ids(lemma = lemma_, searchLang = "EN", pos = pos_)
  print(ids)
  glosses = []

  for id in ids:
    synset = api.get_synset(id = id["id"])

    # We do not want Named entities and idk what not TODO: do we want anything else?
    if (synset["synsetType"] != "CONCEPT"):
      continue
    # Extracting gloss is not easy
    # many glosses from many sources
    # What about language!
    # Currently choosing the first gloss
    # TODO make a function
    try:
      gloss = synset["glosses"][0]["gloss"]
    except IndexError:
      # print(synset)
      continue

    glosses.append(gloss)
    print(gloss)

  return glosses

In [ ]:
# debug above function
glosses = babelnet_glosses("resolution", "NOUN", api)
# print(glosses)

[{'id': 'bn:00025735n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:00067340n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:00034733n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:00067342n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:17148721n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:21398977n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:00799101n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:21117885n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:03598364n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:00067343n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:16865017n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:17620577n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:02084347n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:02870837n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:14856724n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:01428867n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:02720621n', 

In [ ]:
from torch.nn import Softmax
soft_m = Softmax(dim = 1)
# takes in model, and the two sentences and returns cls score
# TODO can switch sent1 and sent2? Score same or different
def sentence_scorer(model, tokensizer, sentence1, sentence2):
  # prepare input to the model 

  encoded_dict = tokenizer(sentence1, sentence2, return_tensors="pt")
  # decoded = tokenizer.decode(encoded_dict["input_ids"])
  # print(decoded)

  with torch.no_grad():
    output = model(**encoded_dict)
    # Is softmax just a computational overhead?
    labels = soft_m(output.logits)

  print(labels[0][0].item())
  # return probability of True
  return labels[0][0].item()

  # in_sent = "[CLS] "+ sentence1 +" [SEP] " + sentence2
  # tokenized_text = tokenizer.tokenize(in_sent)

  # indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)


In [ ]:
sent1 = "The griffon type is characterized by rough - or wire - hair ."
sent2 = "There are three lines of the griffon type recognized by the Fédération Cynologique Internationale ( FCI ) : the Griffon Vendéens , the wirehaired pointers , and the smousje ( Belgian companion dogs or Dutch Smoushond ) ."
sent3 = "Ability to detect vertical movements , variations and dislocations of ground areas to a resolution of a few centimetres ;"
sent4 = "Although Iraq was fully implementing all United Nations resolutions , it continued to suffer the devastating effects of the sanctions ."
sent5 = "When a wise king is heedful of virtue , foreigners from all quarters come as guests ."
sent6 = "He led a virtuous life , and was heedful of the poor and sick , all while attending to his pastoral duties and his own spiritual life ."
print(sentence_scorer(model, tokenizer, sent5, sent6))

0.9774248600006104
0.9774248600006104


In [ ]:
# function to score the glosses agaisnt a sentence
def gloss_score(model, tokenizer, sentence, glosses):
  scores = []

  for gloss in glosses:
    score = sentence_scorer(model,tokenizer, sentence, gloss)
    scores.append(score)

  return scores

In [ ]:
lemma = "resolution"
glosses = babelnet_glosses(lemma, "NOUN", api)

[{'id': 'bn:00025735n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:00067340n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:00034733n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:00067342n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:17148721n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:21398977n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:00799101n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:21117885n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:03598364n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:00067343n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:16865017n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:17620577n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:02084347n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:02870837n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:14856724n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:01428867n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:02720621n', 

KeyboardInterrupt: ignored

In [ ]:
lemma2 = "heedful"
glosses2 = babelnet_glosses(lemma2, "ADJ", api)

[{'id': 'bn:00096547a', 'pos': 'ADJ', 'source': 'BABELNET'}, {'id': 'bn:00097726a', 'pos': 'ADJ', 'source': 'BABELNET'}, {'id': 'bn:00099269a', 'pos': 'ADJ', 'source': 'BABELNET'}]
Giving attention
Taking heed; giving close and thoughtful attention
Cautiously attentive


In [ ]:
scores1 = gloss_score(model,tokenizer, sent5, glosses2)
scores2 = gloss_score(model, tokenizer, sent6, glosses2)
for i,score in enumerate(scores1):
  print(score, end = "\t")
  print(scores2[i])
  print(glosses2[i])

print(glosses2[scores1.index(max(scores1))])
print(glosses2[scores2.index(max(scores2))])

0.21681834757328033
0.9803824424743652
0.24922695755958557
0.028664710000157356
0.9592226147651672
0.20078033208847046
0.21681834757328033	0.028664710000157356
Giving attention
0.9803824424743652	0.9592226147651672
Taking heed; giving close and thoughtful attention
0.24922695755958557	0.20078033208847046
Cautiously attentive
Taking heed; giving close and thoughtful attention
Taking heed; giving close and thoughtful attention
